In [1]:
import numpy as np
import random
import pandas as pd
import pickle
import torch
from torch import nn
from lenskit.algorithms.als import BiasedMF
from lenskit.batch import predict
from lenskit.metrics.predict import rmse

In [2]:
train = pd.read_pickle("../data/ml-1m-split/train.pkl")
test = pd.read_pickle("../data/ml-1m-split/test.pkl")
full = pd.read_pickle("../data/ml-1m-split/full.pkl")

num_users = len(full.groupby("user").size())
num_items = len(full.groupby("item").size())

In [26]:
test

,user,item,item_id,rating,timestamp
30,0,2102,2294,4,978824291
18,0,2889,3105,5,978301713
32,0,1439,1566,4,978824330
50,0,2898,3114,4,978302174
52,0,1154,1246,4,978302091
...,...,...,...,...,...
1000098,6039,1499,1633,4,960972693
1000067,6039,690,722,3,960971992
999922,6039,3217,3449,3,956715966
999965,6039,2885,3101,4,956715865


In [3]:
train_u = train.sort_values(by="user", axis=0).reset_index(drop=True).drop(["timestamp"], axis=1).to_numpy()
train_m = train.sort_values(by="item", axis=0).reset_index(drop=True).drop(["timestamp"], axis=1).to_numpy()
test_u = test.sort_values(by="user", axis=0).reset_index(drop=True).drop(["timestamp"], axis=1).to_numpy()
U_freqs = train.groupby("user").size().values
U_freqs_test = test.groupby("user").size().values
# M_freqs = train.groupby("item").size().values
M_group = train.groupby("item").size()
M_freqs = np.zeros(num_items, dtype="int")
for i in M_group.index:
  M_freqs[i] = M_group[i]
  
U_start = np.cumsum(U_freqs)
M_start = np.cumsum(M_freqs)
U_start = np.insert(U_start, 0, 0)
M_start = np.insert(M_start, 0, 0)

U_start_test = np.cumsum(U_freqs_test)

In [23]:
len(U_start_test)

6040

In [97]:
train[train["item"] == 0]

,user,item,item_id,rating,timestamp
40,0,0,1,5,978824268
581,7,0,1,4,978233496
711,8,0,1,5,978225952
837,9,0,1,5,978226474
1966,17,0,1,4,978154768
...,...,...,...,...,...
997248,6021,0,1,5,956755763
997541,6024,0,1,5,956812867
998170,6031,0,1,4,956718127
998360,6034,0,1,4,956712849


In [90]:
train.sort_values(by="item", axis=0, ascending=False)

,user,item,rating,timestamp
267462,1625,3952,1,974721922
24157,172,3952,2,979251907
357162,2091,3952,4,987372424
495313,3040,3952,4,985655336
169881,1078,3952,4,984260476
...,...,...,...,...
554352,3410,1,3,967757354
765704,4559,1,3,964813020
486490,2994,1,4,970696738
486833,2995,1,5,972416116


In [84]:
train[train["item"] == 1193]

,user,item,rating,timestamp
0,0,1193,5,978300760
120,1,1193,5,978298413
1339,11,1193,4,978220179
1518,14,1193,4,978199279
1747,16,1193,5,978158471
...,...,...,...,...
998284,6032,1193,5,956713500
998423,6034,1193,5,956710879
998894,6035,1193,5,956710766
999580,6036,1193,4,956709215


In [112]:
train["item"] = train["item_id"]

In [62]:
train = train.drop(["item_id"], axis=1)

In [130]:
train

,user,item,item_id,rating,timestamp
0,0,1193,1193,5,978300760
1,0,661,661,3,978302109
3,0,3408,3408,4,978300275
4,0,2355,2355,5,978824291
5,0,1197,1197,3,978302268
...,...,...,...,...,...
1000204,6039,1091,1091,1,956716541
1000205,6039,1094,1094,5,956704887
1000206,6039,562,562,5,956704746
1000207,6039,1096,1096,4,956715648


In [135]:
test["item"] = test["item_id"]

In [136]:
test

,user,item,item_id,rating,timestamp
30,0,2294,2294,4,978824291
18,0,3105,3105,5,978301713
32,0,1566,1566,4,978824330
50,0,3114,3114,4,978302174
52,0,1246,1246,4,978302091
...,...,...,...,...,...
1000098,6039,1633,1633,4,960972693
1000067,6039,722,722,3,960971992
999922,6039,3449,3449,3,956715966
999965,6039,3101,3101,4,956715865


In [4]:
algo = BiasedMF(30, bias=False, method="cd", reg=0.1, damping=5)
algo.fit(train)

In [5]:
preds = predict(algo, test)
overall_rmse = rmse(preds["prediction"], preds["rating"])
overall_rmse

0.8601705563411459

In [66]:
algo.score(0, algo.item_index_.get_indexer(train.groupby("user").get_group(0)["item"]))

array([4.4523959 , 3.20152426, 4.14114247, 3.95893661, 4.15572192,
       4.35416371, 4.42899447, 4.23638159, 4.54856978, 3.82567811,
       4.26650717, 4.13355317, 4.40296126, 4.00224631, 3.72364242,
       4.2351534 , 4.06507132, 3.57988795, 4.08931339, 4.20484891,
       4.68915294, 3.18917242, 4.37969705, 4.10334686, 3.75696731,
       4.18044396, 3.37129753, 3.9183446 , 3.48959625, 3.40454917,
       4.2398087 , 4.19565194, 4.32518474, 4.26354873, 4.23638014,
       3.90175258, 4.41550478, 4.23450307, 4.14289715, 4.41291457,
       3.91607642, 4.12044211])

In [67]:
algo.predict_for_user(0, [1193], ratings=[5])

1193    4.452396
dtype: float64

In [6]:
preds

,user,item,item_id,rating,timestamp,prediction
30,0,2102,2294,4,978824291,3.537587
18,0,2889,3105,5,978301713,4.123646
32,0,1439,1566,4,978824330,3.346799
50,0,2898,3114,4,978302174,4.325284
52,0,1154,1246,4,978302091,4.262889
...,...,...,...,...,...,...
1000098,6039,1499,1633,4,960972693,3.719583
1000067,6039,690,722,3,960971992,3.890019
999922,6039,3217,3449,3,956715966,2.218904
999965,6039,2885,3101,4,956715865,2.941588


In [7]:
U = algo.user_features_
M = algo.item_features_
item_index = algo.item_index_
user_index = algo.user_index_

In [8]:
np.amin(U)

-1.3193057193392776

In [9]:
np.amax(U)

1.761935041812768

In [10]:
algo.item_index_.get_indexer([661])
#np.dot(U[0], M[0])

array([1195])

In [72]:
groups = train.groupby("user").groups
square_error = 0
for group in train.groupby("user"):
  items = item_index.get_indexer(group[1]["item"].values)
#   residuals2 = algo.score(group[0], items) - group[1]["rating"].values
#   square_error += np.sum(residuals ** 2)
  
  uv = U[group[0]]
  im = M[items]
#   print(im.shape)
#   print(uv.shape)
  rv = np.matmul(im, uv)
  residuals = rv - group[1]["rating"].values
#   print(residuals.shape)
#   print(residuals2.shape)
#   assert(np.array_equal(residuals, residuals2))
  square_error += np.sum(residuals ** 2)
  
#   if group[0] > 10:
#     break
  
rmse_value = np.sqrt(square_error / len(train))
rmse_value

0.7837206933214862

In [22]:
def evaluate():
  return np.sqrt(np.mean(np.apply_along_axis(lambda x: (x[2] - np.clip(np.dot(U[x[0]], M[x[1]]), 1, 5)) ** 2, 1, train_u), axis=0))

In [23]:
def eval2():
  square_error = 0
  for (idx, [user, item, _, rating, _]) in train.iterrows():
    square_error += ((rating - np.clip(np.dot(U[user], M[item]), 1, 5)) ** 2)
    if idx % 100000 == 0:
      print(idx)
  return np.sqrt(square_error / len(train))

In [24]:
my_preds = train
item_index = algo.item_index_
user_index = algo.user_index_
my_preds["prediction"] = my_preds.apply(lambda row: np.dot(U[user_index.get_indexer([row["user"]])[0]], M[item_index.get_indexer([row["item_id"]])[0]]), axis=1)

KeyboardInterrupt: 

In [ ]:
my_preds

In [54]:
evaluate()

1.3717658136902975

In [47]:
eval2()

0
100000


KeyboardInterrupt: 

In [37]:
algo.user_features_.shape

(6040, 30)

In [22]:
train_items = train.sort_values(by="item", axis=0)["item"].drop_duplicates().values

In [52]:
len(M_start)

3707

In [7]:
U_freqs

array([ 42, 103,  41, ...,  16,  98, 273])

In [8]:
U_start[9]

639

In [79]:
def get_u_step(user):
    return np.mean(np.apply_along_axis(lambda x: (x[2] - np.dot(U[user[0]], M[x[1]])) * M[x[1]], 1, train_u[U_start[user[0]] : U_start[user[0] + 1]]), axis=0)

def get_m_step(item):
    return np.mean(np.apply_along_axis(lambda x: (x[2] - np.dot(U[x[0]], M[item[0]])) * U[x[0]], 1, train_m[M_start[item[0]] : M_start[item[0] + 1]]), axis=0)
     

In [97]:
%%time
num_factors = 30
lrate = 0.1
U = np.random.uniform(0, 0.6, (num_users, num_factors))
M = np.random.uniform(0, 0.6, (num_items, num_factors))

lrate * np.apply_along_axis(get_u_step, 1, np.arange(num_users).reshape(-1, 1))

CPU times: user 7.8 s, sys: 81 ms, total: 7.88 s
Wall time: 7.89 s


array([[0.05163281, 0.05683624, 0.04578771, ..., 0.05079945, 0.04549282,
        0.05502963],
       [0.04091203, 0.03370367, 0.03911205, ..., 0.04094239, 0.04413883,
        0.03814387],
       [0.04337609, 0.04666317, 0.04887803, ..., 0.04839563, 0.05177086,
        0.05284117],
       ...,
       [0.028283  , 0.02849512, 0.03034026, ..., 0.03234908, 0.02590766,
        0.02952698],
       [0.02714287, 0.03094314, 0.02730581, ..., 0.0299673 , 0.02974747,
        0.02918572],
       [0.01292772, 0.01016208, 0.01062782, ..., 0.00978245, 0.01192125,
        0.00985399]])

In [98]:
%%time
num_factors = 30
lrate = 0.1
U = np.random.uniform(0, 0.6, (num_users, num_factors))
M = np.random.uniform(0, 0.6, (num_items, num_factors))


for user in range(num_users):
  step = get_u_step([user])
  U[user] += lrate * step
  # step_size += np.sum(lrate * np.abs(step))

CPU times: user 7.5 s, sys: 75.7 ms, total: 7.57 s
Wall time: 7.58 s


In [101]:
num_factors = 30
lrate = 0.1
U = np.random.uniform(0, 0.6, (num_users, num_factors))
M = np.random.uniform(0, 0.6, (num_items, num_factors)) 

def evaluate():
  return np.sqrt(np.mean(np.apply_along_axis(lambda x: (x[2] - np.dot(U[x[0]], M[x[1]])) ** 2, 1, train_u), axis=0))

%time evaluate()

CPU times: user 6.67 s, sys: 72.6 ms, total: 6.74 s
Wall time: 6.76 s


1.4739153337402944

In [102]:
def alt_min(num_factors, lrate):
    U = np.random.uniform(0, 0.6, (num_users, num_factors))
    M = np.random.uniform(0, 0.6, (num_items, num_factors))
    
    def evaluate():
      return np.sqrt(np.mean(np.apply_along_axis(lambda x: (x[2] - np.dot(U[x[0]], M[x[1]])) ** 2, 1, train_u), axis=0))
    
    rmse = evaluate()
    prev_rmse = 0
    outer = 0
    
    print("Initial RMSE: {}\n".format(rmse))
    
    def get_u_step(user):
      return np.mean(np.apply_along_axis(lambda x: (x[2] - np.dot(U[user], M[x[1]])) * M[x[1]], 1, train_u[U_start[user] : U_start[user + 1]]), axis=0)

    def get_m_step(item):
      return np.mean(np.apply_along_axis(lambda x: (x[2] - np.dot(U[x[0]], M[item])) * U[x[0]], 1, train_m[M_start[item] : M_start[item + 1]]), axis=0)
         
    while abs(rmse - prev_rmse) > 0.001:
        prev_rmse = rmse
        
        # Optmize U
        step_size = 100
        inner = 0
        while abs(step_size) > 0.001:
            step_size = 0
            for user in range(num_users):
              step = get_u_step(user)
              U[user] += lrate * step
              step_size += np.sum(lrate * np.abs(step))
            
            step_size /= (num_users * num_factors)
            
            print("USER ITER {}\nStep size: {}\nRMSE: {}\n".format(inner, step_size, evaluate()))
            inner += 1
            
        
        # Optimize M
        step_size = 100
        inner = 0
        while abs(step_size) > 0.001:            
            step_size = 0
            for item in M_group.index:
              step = get_m_step(item)
              M[item] += lrate * step
              step_size += np.sum(lrate * np.abs(step))
            
            step_size /= (num_items * num_factors)
            
            print("ITEM ITER {}\nStep size: {}\nRMSE: {}\n".format(inner, step_size, evaluate()))
            inner += 1

        rmse = evaluate()

        print("\n============ ROUND {} ============\nRMSE: {}\nPrev RMSE: {}\nDiff: {}\n".format(outer, rmse, prev_rmse, rmse - prev_rmse))
        outer += 1

In [103]:
alt_min(30, 0.1)

Initial RMSE: 1.4886325132903389

USER ITER 0
Step size: 0.02973182459523753
RMSE: 1.3235270196389872



KeyboardInterrupt: 

In [59]:
num_factors = 30
epochs = 1

def alt_min(num_factors, learning_rate):
  model = MatrixFactorization(num_users, num_items, num_factors)
  params = list(model.parameters())
  user_optimizer = torch.optim.SGD([params[0]], lr=learning_rate)
  item_optimizer = torch.optim.SGD([params[1]], lr=learning_rate)
  loss_fn = nn.MSELoss()

  for i in range(epochs):
    for (idx, [user, item, _, rating, _]) in train.iterrows():
      prediction = model(torch.LongTensor([user]), torch.LongTensor([item]))
      loss = loss_fn(prediction, torch.FloatTensor([rating]))
      grad = loss.backward()
      print(grad)
      user_optimizer.step()
      model.zero_grad()

In [25]:
avg_item_ratings = np.zeros(num_items)
for (item, df) in train.groupby("item"):
  avg_item_ratings[item] = np.mean(df["rating"].values)

In [33]:
train_shuffled = train.sample(frac=1).reset_index(drop=True)

In [36]:
len(train_shuffled)

800193

In [58]:
class MatrixFactorization(nn.Module):
  def __init__(self, num_users, num_items, num_factors):
    super().__init__()
    self.user_factors = nn.Embedding(num_users, num_factors, sparse=False)
    self.item_factors = nn.Embedding(num_items, num_factors, sparse=False)
    
    self.user_factors.weight.data.uniform_(-0.25, 0.25)
    self.item_factors.weight.data.uniform_(-0.25, 0.25)
    
  def forward(self, users, items):
    return torch.diagonal(torch.mm(self.user_factors(users), torch.transpose(self.item_factors(items), 0, 1)))

In [40]:
epochs = 10

def alt_min(num_factors, lrate):
  M = np.random.uniform(0, 0.25, (num_items, num_factors))
  # M[:, 0] = avg_item_ratings
  U = np.ones((num_users, num_factors))
  U_prev = np.zeros((num_users, num_factors))
  U_freqs = train.groupby("user").size().values.reshape(-1, 1)    
    
  for i in range(epochs):
    U_step = np.zeros((num_users, num_factors))
    total_loss = 0
    for (idx, [user, item, _, rating, _]) in train_shuffled.iterrows():
      residual = rating - np.dot(U[user], M[item])
      total_loss += residual ** 2
      U_step[user] += residual * M[item]
    
    U_step = lrate * (U_step / U_freqs)
    U += U_step
    print("EPOCH 1\nStep L2 Norm: {}\nRMSE: {}".format(np.linalg.norm(U_step), np.sqrt(total_loss / len(train_shuffled))))
    
          
      
    
  
  

In [43]:
alt_min(30, 0.5)

0
200000
400000
600000
800000
EPOCH 1
Step L2 Norm: 12.253237942641325
RMSE: 1.2000548624106067
0
200000
400000
600000
800000
EPOCH 1
Step L2 Norm: 9.43106566112119
RMSE: 1.1570828266019815
0
200000
400000
600000
800000
EPOCH 1
Step L2 Norm: 7.318747459406452
RMSE: 1.1311688881735902
0
200000
400000
600000
800000
EPOCH 1
Step L2 Norm: 5.752674947704704
RMSE: 1.1156156609148236
0
200000
400000
600000
800000
EPOCH 1
Step L2 Norm: 4.6083465600404665
RMSE: 1.1062456775370202
0
200000


KeyboardInterrupt: 